In [1]:
import torch
import torchvision
from torch.utils.data import DataLoader
from torchvision.models.detection import FasterRCNN
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
from torchvision.datasets import CocoDetection
from torchvision.transforms import functional as F
from eval import evaluate
from tqdm import tqdm
from itertools import product
import time

In [2]:

class CocoTransform:
    def __call__(self, image, target):
        image = F.to_tensor(image)

        boxes = []
        labels = []

        for obj in target:
            bbox = obj["bbox"]
            # COCO: [x, y, width, height] → [x1, y1, x2, y2]
            x, y, w, h = bbox
            boxes.append([x, y, x + w, y + h])
            labels.append(obj["category_id"])

        boxes = torch.tensor(boxes, dtype=torch.float32)
        labels = torch.tensor(labels, dtype=torch.int64)

        target_converted = {
            "boxes": boxes,
            "labels": labels
        }

        return image, target_converted

In [3]:

def get_coco_dataset(img_dir, ann_file):
    return CocoDetection(
        root=img_dir,
        annFile=ann_file,
        transforms=CocoTransform()
    )


train_dataset = get_coco_dataset(
    img_dir="xray_knifes-6/train",
    ann_file="xray_knifes-6/train/_annotations.coco.json"
)


val_dataset = get_coco_dataset(
    img_dir="xray_knifes-6/valid",
    ann_file="xray_knifes-6/valid/_annotations.coco.json"
)




train_loader = DataLoader(train_dataset, batch_size=18, shuffle=True, collate_fn=lambda x: tuple(zip(*x)))
val_loader = DataLoader(val_dataset, batch_size=18, shuffle=False, collate_fn=lambda x: tuple(zip(*x)))

loading annotations into memory...
Done (t=0.09s)
creating index...
index created!
loading annotations into memory...
Done (t=0.01s)
creating index...
index created!


In [4]:

def get_model(num_classes):

    model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)

    in_features = model.roi_heads.box_predictor.cls_score.in_features
    
    model.roi_heads.box_predictor = FastRCNNPredictor(in_features, num_classes)
    return model

In [5]:

num_classes = 9 
model = get_model(num_classes)

c:\Users\bilgi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\bilgi\AppData\Local\Programs\Python\Python311\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=FasterRCNN_ResNet50_FPN_Weights.COCO_V1`. You can also use `weights=FasterRCNN_ResNet50_FPN_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [6]:

device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
model.to(device)

params = [p for p in model.parameters() if p.requires_grad]
optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.0005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

In [7]:
def train_one_epoch(model, optimizer, data_loader, device, epoch):
    model.train()
    total_loss = 0.0
    num_batches = len(data_loader)


    pbar = tqdm(enumerate(data_loader), total=num_batches, desc=f"Epoch {epoch}", ncols=100)

    for batch_idx, (images, targets) in pbar:
        images = list(img.to(device) for img in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]

        loss_dict = model(images, targets)
        losses = sum(loss for loss in loss_dict.values())
        loss_value = losses.item()
        total_loss += loss_value

        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        
        # Progress bar'ı güncelle
        pbar.set_postfix({
            "Loss": f"{loss_value:.4f}"
        })

    avg_loss = total_loss / num_batches
    print(f"✅ Epoch {epoch} tamamlandı. Ortalama Loss: {avg_loss:.4f}")

    

In [8]:
def run_training(model, optimizer, lr_scheduler, num_epochs):
    for epoch in range(num_epochs):
        train_one_epoch(model, optimizer, train_loader, device, epoch)
        lr_scheduler.step()


        if (epoch + 1) % 5 == 0:
            metrics = evaluate(model, val_loader, device, num_classes)
            print("\n📊 Evaluation Results:")
            print(f"mAP@50: {metrics['mAP_50']:.4f}")
            print(f"mAP@75: {metrics['mAP_75']:.4f}")

        model_path = f"fasterrcnn_resnet50_epoch_{epoch + 1}.pth"
        torch.save(model.state_dict(), model_path)
        print(f"💾 Model saved: {model_path}")

In [ ]:
def run_grid_search():


    lr_list = [0.005, 0.0025, 0.001]
    momentum_list = [0.9]
    weight_decay_list = [0.0001, 0.00005]


    param_grid = list(product(lr_list, momentum_list, weight_decay_list))
    best_map50 = 0.0
    best_params = None

    for idx, (lr, momentum, wd) in enumerate(param_grid):
        print(f"\n🔍 [Grid {idx+1}/{len(param_grid)}] lr={lr}, momentum={momentum}, weight_decay={wd}")

        model = get_model(num_classes).to(device)
        params = [p for p in model.parameters() if p.requires_grad]
        optimizer = torch.optim.SGD(params, lr=lr, momentum=momentum, weight_decay=wd)
        lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=3, gamma=0.1)

        train_one_epoch(model, optimizer, train_loader, device, epoch=0)
        lr_scheduler.step()

        metrics = evaluate(model, val_loader, device, num_classes)
        map_50 = metrics['mAP_50']
        print(f"🧪 Eval mAP@50: {map_50:.4f}")

        if map_50 > best_map50:
            best_map50 = map_50
            best_params = (lr, momentum, wd)

    print("\n🏆 En iyi hiperparametreler:")
    print(f"Learning rate: {best_params[0]}")
    print(f"Momentum: {best_params[1]}")
    print(f"Weight decay: {best_params[2]}")
    print(f"Best mAP@50: {best_map50:.4f}")

    lr = best_params[0]
    momentum = best_params[1]
    w_d = best_params[2]
    return lr, momentum, w_d

: 

In [ ]:
model = get_model(num_classes).to(device)
params = [p for p in model.parameters() if p.requires_grad]

optimizer = torch.optim.SGD(params, lr=0.005, momentum=0.9, weight_decay=0.00005)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.1)


run_training(model, optimizer, lr_scheduler, num_epochs=100)


Epoch 0: 100%|███████████████████████████████████████| 614/614 [08:17<00:00,  1.23it/s, Loss=0.2312]


✅ Epoch 0 tamamlandı. Ortalama Loss: 0.2846
💾 Model saved: fasterrcnn_resnet50_epoch_1.pth


Epoch 1: 100%|███████████████████████████████████████| 614/614 [08:31<00:00,  1.20it/s, Loss=0.2286]


✅ Epoch 1 tamamlandı. Ortalama Loss: 0.2053
💾 Model saved: fasterrcnn_resnet50_epoch_2.pth


Epoch 2: 100%|███████████████████████████████████████| 614/614 [08:55<00:00,  1.15it/s, Loss=0.2555]


...

Epoch 34: 100%|██████████████████████████████████████| 614/614 [07:49<00:00,  1.31it/s, Loss=0.0959]


✅ Epoch 34 tamamlandı. Ortalama Loss: 0.0572


Evaluating: 100%|██████████| 157/157 [01:05<00:00,  2.38it/s]



📊 Evaluation Results:
mAP@50: 0.7331
mAP@75: 0.3453
💾 Model saved: fasterrcnn_resnet50_epoch_35.pth


Epoch 35:  30%|███████████▍                          | 184/614 [02:18<05:22,  1.33it/s, Loss=0.0604]